In [ ]:
# Using Julia 0.6.2

We will look at the IterativeSolvers package that we have seen in class.
I want to see how it works and whether it is already useful for research

I will focus on testing GMRES

In [13]:
Pkg.add("IterativeSolvers")
Pkg.update("IterativeSolvers")

INFO: Package IterativeSolvers is already installed
INFO: Updating METADATA...


LoadError: [91mGitError(Code:ENOTFOUND, Class:Reference, Revspec 'HEAD' not found.)[39m

In [15]:
using IterativeSolvers

Let's create some matrix to test on

In [16]:
# simple function to get a matrix for poisson equation discretization with constant coefficient
# this function was tested for 0 based indexing and then modified so I don't vouch for it
# but since I want to test GMRES it doesn't matter that much
function getfun(n)
    A = spzeros(n * n,n * n)
    for i in 1:(n*n)
        A[i,i] = 4
        # set the corners
        if i == 1
            A[i,n + 1] = -1
            A[i,2] = -1
        elseif i == n
            A[i,n-1] = -1
            A[i,2*n] = -1
        elseif i == (n-1) * n + 1
            A[i, (n-2)*n + 1] = -1
            A[i, (n-1)*n + 2] = -1
        elseif i == n * n
            A[i, n * n - 1] = -1
            A[i, (n-1) * n] = -1
        #set the upper and lower borders
        elseif i > 1 && i < n
            A[i,i-1] = -1
            A[i,i+1] = -1
            A[i,i+n] = -1
        elseif i > (n-1) * n + 1 && i < n * n
            A[i,i-1] = - 1
            A[i,i+1] = - 1
            A[i,i-n] = -1
        # set the left and right borders
        elseif i % n == 1
            A[i, i - n] = -1
            A[i, i + n] = -1
            A[i, i + 1] = -1
        elseif i % n == 0
            A[i,i-n] = -1
            A[i,i+n] = -1
            A[i,i-1] = -1
        else
            A[i,i+1]= -1
            A[i,i-1]= -1
            A[i,i-n]= -1
            A[i,i+n]= -1
        end
    end
    return A
end

getfun (generic function with 1 method)

In [17]:
n = 100
A = getfun(n)
# the matrix dimension is 10,000 x 10,000 (sparse), not large but should be enough to test functionalities

10000×10000 SparseMatrixCSC{Float64,Int64} with 49600 stored entries:
  [1    ,     1]  =  4.0
  [2    ,     1]  =  -1.0
  [101  ,     1]  =  -1.0
  [1    ,     2]  =  -1.0
  [2    ,     2]  =  4.0
  [3    ,     2]  =  -1.0
  [102  ,     2]  =  -1.0
  [2    ,     3]  =  -1.0
  [3    ,     3]  =  4.0
  [4    ,     3]  =  -1.0
  ⋮
  [9898 ,  9998]  =  -1.0
  [9997 ,  9998]  =  -1.0
  [9998 ,  9998]  =  4.0
  [9999 ,  9998]  =  -1.0
  [9899 ,  9999]  =  -1.0
  [9998 ,  9999]  =  -1.0
  [9999 ,  9999]  =  4.0
  [10000,  9999]  =  -1.0
  [9900 , 10000]  =  -1.0
  [9999 , 10000]  =  -1.0
  [10000, 10000]  =  4.0

Now we will test GMRES on the preconditioned system

In [18]:
# constant solution for poisson equation is difficult for GMRES on large systems
x = ones(n * n)
b = A * x

10000-element Array{Float64,1}:
 2.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 2.0

We will use gmres function.

Docs say that the function returns the whole history, we'll see how useful it is (scipy returns residual at each step during execution, if promted).

It is not clear from documentation whether one can turn off restarting but one can set it to a large number of iterations.

The package works with sparse matrices.

First, we will see if maxiter works

In [19]:
# we will assume the initial x is zero
xg, hist = gmres(A, b, tol = 1e-6, maxiter = 5, log = true)
@show norm(xg - x)/norm(x)
@show hist

norm(xg - x) / norm(x) = 0.9542466502550678
hist = Not converged after 5 iterations.


Not converged after 5 iterations.

In [20]:
# we will assume the initial x is zero
xg, hist = gmres(A, b, tol = 1e-6, maxiter = 15, log = true)
@show norm(xg - x)/norm(x)
@show hist

norm(xg - x) / norm(x) = 0.8802482874662134
hist = Not converged after 15 iterations.


Not converged after 15 iterations.

In [21]:
# we will assume the initial x is zero
xg, hist = gmres(A, b, tol = 1e-6, maxiter = 50, log = true)
@show norm(xg - x)/norm(x)
@show hist

norm(xg - x) / norm(x) = 0.65943304877824
hist = Not converged after 50 iterations.


Not converged after 50 iterations.

In [22]:
# we will assume the initial x is zero
xg, hist = gmres(A, b, tol = 1e-6, maxiter = 150, log = true)
@show norm(xg - x)/norm(x)
@show hist

norm(xg - x) / norm(x) = 0.2647130235799437
hist = Not converged after 150 iterations.


Not converged after 150 iterations.

maxiter appears to be working.

History is nice. You can get the history of residuals, number of matrix-vectors products, etc.
(although this is not well-documented yet, or not linked to gmres documentation, 
    you have to look into the implementation to see what's there)

In [23]:
# number of matrix-vector products
@show hist.mvps
# number of iterations
@show hist.iters
# data about residuals
@show hist.data[:resnorm]

hist.mvps = 159
hist.iters = 150
hist.data[:resnorm] = [9.10055, 5.68281, 4.33595, 3.32493, 2.71659, 2.24876, 1.91444, 1.65071, 1.4446, 1.27803, 1.14069, 1.02734, 0.930534, 0.849176, 0.777976, 0.717204, 0.663057, 0.616225, 0.573925, 0.536921, 0.518979, 0.500536, 0.481693, 0.462529, 0.443174, 0.423715, 0.404289, 0.385016, 0.366005, 0.347454, 0.32938, 0.312131, 0.295542, 0.280207, 0.265645, 0.252834, 0.240776, 0.231014, 0.221776, 0.21536, 0.212199, 0.20904, 0.206103, 0.202032, 0.196744, 0.191712, 0.186261, 0.181448, 0.176477, 0.171805, 0.167212, 0.162823, 0.158687, 0.154755, 0.151091, 0.147628, 0.144397, 0.141331, 0.138457, 0.135684, 0.133654, 0.131838, 0.130031, 0.12804, 0.125844, 0.123685, 0.121577, 0.11972, 0.117944, 0.116335, 0.114881, 0.113494, 0.112195, 0.110714, 0.109321, 0.107953, 0.106896, 0.106074, 0.105188, 0.104329, 0.103612, 0.102906, 0.10225, 0.101372, 0.100236, 0.0991423, 0.0980326, 0.0971149, 0.096176, 0.0952451, 0.0942719, 0.0932714, 0.092245, 0.0911198, 0.0899871, 0.088

150-element Array{Float64,1}:
 9.10055  
 5.68281  
 4.33595  
 3.32493  
 2.71659  
 2.24876  
 1.91444  
 1.65071  
 1.4446   
 1.27803  
 1.14069  
 1.02734  
 0.930534 
 ⋮        
 0.0585354
 0.0579012
 0.0573171
 0.0567495
 0.0561388
 0.0555178
 0.0548819
 0.0542659
 0.053669 
 0.0531154
 0.0525915
 0.0520926

Now we will try to use ILU as a preconditioner. The package itself doesn't have it, but recommends ILU from the following package. We'll try to make it work.

In [24]:
Pkg.clone("git@github.com:haampie/ILU.jl.git")

INFO: Cloning ILU from git@github.com:haampie/ILU.jl.git


LoadError: [91mILU already exists[39m

In [25]:
Pkg.add("ILU")

INFO: Package ILU is already installed


In [26]:
using ILU

LoadError: [91mArgumentError: Module ILU not found in current path.
Run `Pkg.add("ILU")` to install the ILU package.[39m

I have installed the package ILU:

INFO: Package ILU is already installed

But I get the error:

ArgumentError: Module ILU not found in current path.
Run `Pkg.add("ILU")` to install the ILU package.

Not sure how to fix it. Anyway, from the docs, it seems that it should be easy to write one's own preconditioner.